In [47]:
import os
import chardet

def detect_file_encoding(file_path):
    """Detects the encoding of a file."""
    with open(file_path, 'rb') as f:
        raw_data = f.read(10000)  # Read a sample of the file
    result = chardet.detect(raw_data)
    return result['encoding']

def extract_log_lines(input_file_path):
    """Extracts lines containing specific substrings from a log file."""
    # Create the subfolder if it doesn't exist
    subfolder = os.path.join(os.path.dirname(input_file_path), 'cleaned_logs')
    os.makedirs(subfolder, exist_ok=True)

    # Create the output file path
    base_name = os.path.basename(input_file_path)
    file_name, file_extension = os.path.splitext(base_name)
    output_file_path = os.path.join(subfolder, f"{file_name}_cleaned{file_extension}")

    # Detect file encoding
    encoding = detect_file_encoding(input_file_path)
    print(f"Detected encoding: {encoding}")

    # Open the input and output files with detected encoding
    try:
        with open(input_file_path, 'r', encoding=encoding, errors='replace') as infile, \
             open(output_file_path, 'w', encoding=encoding) as outfile:
            for line in infile:
                if "Processing row:" in line or "Direct LLM Response" in line:
                    outfile.write(line)
        print(f"Cleaned log saved to: {output_file_path}")
    except UnicodeDecodeError as e:
        print(f"Error reading the file: {e}")

    return output_file_path

In [1]:
import os
import chardet

def detect_file_encoding(file_path):
    """Detects the encoding of a file."""
    with open(file_path, 'rb') as f:
        raw_data = f.read(10000)  # Read a sample of the file
    result = chardet.detect(raw_data)
    return result['encoding']

def extract_log_lines(input_file_path):
    """Extracts lines containing specific substrings from a log file."""
    # Create the subfolder if it doesn't exist
    subfolder = os.path.join(os.path.dirname(input_file_path), 'cleaned_logs')
    os.makedirs(subfolder, exist_ok=True)

    # Create the output file path
    base_name = os.path.basename(input_file_path)
    file_name, file_extension = os.path.splitext(base_name)
    output_file_path = os.path.join(subfolder, f"{file_name}_cleaned{file_extension}")

    # Detect file encoding
    encoding = detect_file_encoding(input_file_path)
    print(f"Detected encoding: {encoding}")

    # Open the input and output files with detected encoding
    try:
        with open(input_file_path, 'r', encoding=encoding, errors='replace') as infile, \
             open(output_file_path, 'w', encoding=encoding) as outfile:
            for line in infile:
                if "Processing row:" in line or "Direct LLM Response" in line:
                    outfile.write(line)
        print(f"Cleaned log saved to: {output_file_path}")
    except UnicodeDecodeError as e:
        print(f"Error reading the file: {e}")

    return output_file_path

In [2]:


extract_log_lines(r"C:\Users\aboeck\Documents\ICMT\MASTERARBEIT_DI\backup_03_07\EVALUATION\log_files\MIXTRAL\_EN_reply_classify_SHORT.log")

Detected encoding: Windows-1252
Cleaned log saved to: C:\Users\aboeck\Documents\ICMT\MASTERARBEIT_DI\backup_03_07\EVALUATION\log_files\MIXTRAL\cleaned_logs\_EN_reply_classify_SHORT_cleaned.log


'C:\\Users\\aboeck\\Documents\\ICMT\\MASTERARBEIT_DI\\backup_03_07\\EVALUATION\\log_files\\MIXTRAL\\cleaned_logs\\_EN_reply_classify_SHORT_cleaned.log'

In [48]:
import os
import pandas as pd
import re
import chardet
def detect_file_encoding(file_path):
    """Detects the encoding of a file."""
    with open(file_path, 'rb') as f:
        raw_data = f.read(10000)  # Read a sample of the file
    result = chardet.detect(raw_data)
    return result['encoding']

def read_file_with_fallback(file_path, encoding_list):
    """Attempts to read a file with a list of encodings."""
    for encoding in encoding_list:
        try:
            with open(file_path, 'r', encoding=encoding) as file:
                return file.read()
        except UnicodeDecodeError:
            print(f"Failed to decode file with encoding {encoding}. Trying next encoding...")
    raise ValueError(f"All encoding attempts failed for file: {file_path}")



def extract_log_data(log_file_path, encoding):
    """Extracts log data using a specified encoding."""
    processing_row_pattern = re.compile(r"Processing row: (\d+)")
    classification_pattern = re.compile(r"'Classification':\s*'([A-Za-z]+)'")
    explanation_pattern = re.compile(r"'Explanation':\s*(.+)")

    log_data = {}
    
    try:
        content = read_file_with_fallback(log_file_path, [encoding, 'utf-8', 'latin-1', 'windows-1252'])
        lines = content.splitlines()
        
        current_row = None
        for line in lines:
            processing_match = processing_row_pattern.search(line)
            if processing_match:
                current_row = int(processing_match.group(1))
                log_data[current_row] = {"classification": None, "explanation": None}
            if current_row is not None:
                classification_match = classification_pattern.search(line)
                if classification_match:
                    log_data[current_row]["classification"] = classification_match.group(1).strip()
                explanation_match = explanation_pattern.search(line)
                if explanation_match:
                    log_data[current_row]["explanation"] = explanation_match.group(1).strip()
    except ValueError as e:
        print(e)
        raise

    return log_data



def clean_classification_value(value):
    """Cleans classification value to ensure it is valid."""
    allowed_classes = {"appropriate", "offensive", "violent", "inappropriate"}
    if isinstance(value, str):
        cleaned_value = value.strip().lower()  # Strip and convert to lowercase
        print(f"Original value: '{value}' | Cleaned value: '{cleaned_value}'")  # Debugging line
        if cleaned_value in allowed_classes:
            return cleaned_value
    return "NONE"

def update_dataframe_with_log_data(df, log_data):
    """Updates DataFrame with log data and captures any errors."""
    error_indices = []
    for index, row in df.iterrows():
        if index in log_data:
            log_entry = log_data[index]
            try:
                # Check if the DataFrame's classification is 'NONE' or missing
                if pd.isna(row["classification"]) or row["classification"].strip().lower() == "none":
                    # Update with log data if valid
                    cleaned_classification = clean_classification_value(log_entry["classification"])
                    if cleaned_classification != "NONE":
                        df.at[index, "classification"] = cleaned_classification
                if pd.isna(row["explanation"]) or row["explanation"].strip() == "":
                    # Update explanation if not present
                    df.at[index, "explanation"] = log_entry["explanation"].strip() if log_entry["explanation"] else ""
            except Exception as e:
                print(f"Error occurred at index {index}: {e}")
                error_indices.append(index)

    df["classification"] = df["classification"].apply(clean_classification_value)
    return df, error_indices


In [49]:
# import os
# import pandas as pd
# import re
# import chardet

# def detect_file_encoding(file_path):
#     """Detects the encoding of a file."""
#     with open(file_path, 'rb') as f:
#         raw_data = f.read(10000)  # Read a sample of the file
#     result = chardet.detect(raw_data)
#     return result['encoding']

# def read_file_with_fallback(file_path, encoding_list):
#     """Attempts to read a file with a list of encodings."""
#     for encoding in encoding_list:
#         try:
#             with open(file_path, 'r', encoding=encoding) as file:
#                 return file.read()
#         except UnicodeDecodeError:
#             print(f"Failed to decode file with encoding {encoding}. Trying next encoding...")
#     raise ValueError(f"All encoding attempts failed for file: {file_path}")

# def extract_log_data(log_file_path, encoding):
#     """Extracts log data using a specified encoding."""
#     processing_row_pattern = re.compile(r"Processing row: (\d+)")
#     classification_pattern = re.compile(r"'Classification':\s*'([A-Za-z]+)'")
#     explanation_pattern = re.compile(r"'Explanation':\s*(.+)")

#     log_data = {}
    
#     try:
#         # Read the file with the detected or fallback encoding
#         content = read_file_with_fallback(log_file_path, [encoding, 'utf-8', 'latin-1', 'windows-1252'])
#         lines = content.splitlines()
        
#         current_row = None
#         for line in lines:
#             processing_match = processing_row_pattern.search(line)
#             if processing_match:
#                 current_row = int(processing_match.group(1))
#                 log_data[current_row] = {"classification": None, "explanation": None}
#             if current_row is not None:
#                 classification_match = classification_pattern.search(line)
#                 if classification_match:
#                     log_data[current_row]["classification"] = classification_match.group(1).strip()
#                 explanation_match = explanation_pattern.search(line)
#                 if explanation_match:
#                     log_data[current_row]["explanation"] = explanation_match.group(1).strip()
#     except ValueError as e:
#         print(e)
#         raise

#     return log_data

# def clean_classification_value(value):
#     """Cleans classification value to ensure it is valid."""
#     allowed_classes = {"appropriate", "offensive", "violent", "inappropriate"}
#     if isinstance(value, str):
#         cleaned_value = value.strip().lower()  # Strip and convert to lowercase
#         print(f"Original value: '{value}' | Cleaned value: '{cleaned_value}'")  # Debugging line
#         if cleaned_value in allowed_classes:
#             return cleaned_value
#     return "NONE"

# def update_dataframe_with_log_data(df, log_data):
#     """Updates DataFrame with log data and captures any errors."""
#     error_indices = []
#     #print("Len logdata: ", len(log_data))
#     for index, row in df.iterrows():
#         #print("INDEX: ", index)
#         #print("log_data: ", log_data)
#         if index in log_data:
#             log_entry = log_data[index]
#             try:
#                 # Check if the DataFrame's classification is 'NONE' or missing
                
#                 #print('row["classification"].strip().lower(): ', row["classification"].strip().lower())


#                 if pd.isna(row["classification"]) or row["classification"].strip().lower() == "none":
#                     # Update with log data if valid
#                     print("INDEX: ", index)
#                     print("log_entry: ", log_entry)
#                     cleaned_classification = clean_classification_value(log_entry["classification"])
#                     if cleaned_classification != "NONE":
#                         df.at[index, "classification"] = cleaned_classification
#                 if pd.isna(row["explanation"]) or row["explanation"].strip() == "":
#                     # Update explanation if not present
#                     df.at[index, "explanation"] = log_entry["explanation"].strip() if log_entry["explanation"] else ""
#             except Exception as e:
#                 print(f"Error occurred at index {index}: {e}")
#                 error_indices.append(index)

#     df["classification"] = df["classification"].apply(clean_classification_value)
#     return df, error_indices

In [52]:
# # Detect encodings
# csv_encoding = detect_file_encoding(csv_file_path)
# log_encoding = detect_file_encoding(log_file_path)

# print(f"Detected CSV encoding: {csv_encoding}")
# print(f"Detected log encoding: {log_encoding}")

# # Read the CSV file
# df = pd.read_csv(csv_file_path, sep=';', encoding=csv_encoding)
# df = df#[:400]

# # Extract log data
# log_data = extract_log_data(log_file_path, encoding=log_encoding)

# # Update DataFrame with log data and capture error indices
# updated_df, error_indices = update_dataframe_with_log_data(df, log_data)

# # Print error indices
# if error_indices:
#     print(f"Errors occurred at indices: {error_indices}")

# # Create the subfolder if it doesn't exist
# subfolder = os.path.join(os.path.dirname(csv_file_path), 'cleaned')
# os.makedirs(subfolder, exist_ok=True)

# # Construct the output file path
# output_csv_file_path = os.path.join(subfolder, os.path.basename(csv_file_path))

# # Save the updated DataFrame to the new CSV file
# updated_df.to_csv(output_csv_file_path, index=False, sep=';', encoding=csv_encoding)

# print(f"Updated DataFrame saved to: {output_csv_file_path}")

# # Set display options to show all rows and full text
# pd.set_option('display.max_rows', None)  # Show all rows

Detected CSV encoding: MacRoman
Detected log encoding: ascii
Failed to decode file with encoding ascii. Trying next encoding...
Failed to decode file with encoding utf-8. Trying next encoding...
INDEX:  4
log_entry:  {'classification': 'Appropriate', 'explanation': "The reply comment is a response to the preceding comment and expresses a differing opinion about the challenges of having children and taking care of them. The comment does not contain any offensive, violent, or inappropriate language and is not directly attacking or insulting the Duchess of Cambridge or any other individual. Therefore, the classification of the reply is 'Appropriate'."}
Original value: 'Appropriate' | Cleaned value: 'appropriate'
INDEX:  23
log_entry:  {'classification': 'Appropriate', 'explanation': "'The comment does not contain any offensive, violent, or inappropriate language. The user is expressing their opinion about the government's approach to handling the virus and comparing it to other countries.

In [42]:
# # Function to read log file content
# def read_log_file(file_path):
#     with open(file_path, 'r') as file:
#         return file.read()


# # Function to extract classification from log file
# def extract_classifications(log_content):
#     pattern = re.compile(r"Processing row: (\d+).*?Direct LLM Response \(No Context\):.*?'Classification': '(\w+)'", re.DOTALL)
#     matches = pattern.findall(log_content)
#     return {int(row): classification.lower() for row, classification in matches}

# # Update DataFrame with log data
# def update_dataframe_with_log_data(df, log_data):
#     extracted_classifications = extract_classifications(log_data)
#     error_indices = []

#     for idx, new_classification in extracted_classifications.items():
#         if df.loc[idx, 'classification'].lower() not in allowed_classes:
#             if new_classification in allowed_classes:
#                 df.loc[idx, 'classification'] = new_classification
#             else:
#                 df.loc[idx, 'classification'] = 'NONE'
#         else:
#             error_indices.append(idx)
    
#     return df, error_indices


In [43]:
# # Paths to the files
# file_name = "EN_reply_comment_title_description_classify_replies_561_MIXTRAL_Hatebase_FINAL"
# csv_directory = "C:/Users/aboeck/Documents/ICMT/MASTERARBEIT_DI/backup_03_07/EVALUATION/experiments_outputs/MIXTRAL/"
# csv_file_path = os.path.join(csv_directory, f'{file_name}.csv')
# log_file_unclean = f"C:/Users/aboeck/Documents/ICMT/MASTERARBEIT_DI/backup_03_07/EVALUATION/log_files/MIXTRAL/_{file_name}.log"
# log_directory = "C:/Users/aboeck/Documents/ICMT/MASTERARBEIT_DI/backup_03_07/EVALUATION/log_files/MIXTRAL/cleaned_logs/"
# log_file_name = f'_{file_name}_cleaned.log'
# log_file_path = os.path.join(log_directory, log_file_name)



# csv_encoding = detect_file_encoding(csv_file_path)
# log_encoding = detect_file_encoding(log_file_path)

# print(f"Detected CSV encoding: {csv_encoding}")
# print(f"Detected log encoding: {log_encoding}")

# # Read the CSV file
# df = pd.read_csv(csv_file_path, sep=';', encoding=csv_encoding)
# df = df#[:400]



# allowed_classes = {"appropriate", "offensive", "violent", "inappropriate", "Appropriate", "Offensive", "Violent", "Inappropriate"}


# # Extract log data
# log_data = extract_log_data(log_file_path, encoding=log_encoding)

# # Update DataFrame with log data and capture error indices
# updated_df, error_indices = update_dataframe_with_log_data(df, log_data)

# # Print error indices
# if error_indices:
#     print(f"Errors occurred at indices: {error_indices}")

# # Create the subfolder if it doesn't exist
# subfolder = os.path.join(os.path.dirname(csv_file_path), 'cleaned')
# os.makedirs(subfolder, exist_ok=True)

# # Construct the output file path
# output_csv_file_path = os.path.join(subfolder, os.path.basename(csv_file_path))


Detected CSV encoding: MacRoman
Detected log encoding: ascii
Failed to decode file with encoding ascii. Trying next encoding...
Failed to decode file with encoding utf-8. Trying next encoding...


TypeError: expected string or bytes-like object

In [57]:
import os
import pandas as pd
import re
from chardet import detect

# Function to detect file encoding
def detect_file_encoding(file_path):
    with open(file_path, 'rb') as f:
        raw_data = f.read()
    return detect(raw_data)['encoding']

# Function to read log file content
def read_log_file(file_path, encoding):
    with open(file_path, 'r', encoding=encoding) as file:
        return file.read()

# Function to extract classification from log file
def extract_classifications(log_content):
    pattern = re.compile(
        r"Processing row: (\d+).*?Direct LLM Response \(No Context\):.*?'?Classification'?:\s*'?(\w+)'?",
        re.DOTALL)
    matches = pattern.findall(log_content)
    return {int(row): classification.lower() for row, classification in matches}

# Update DataFrame with log data
def update_dataframe_with_log_data(df, log_data, allowed_classes):
    extracted_classifications = extract_classifications(log_data)
    error_indices = []

    for idx, new_classification in extracted_classifications.items():
        current_classification = df.loc[idx, 'classification']
        print(f"Processing row {idx}: current classification = '{current_classification}', new classification = '{new_classification}'")

        # Handle missing values and ensure lower() is only called on strings
        if pd.isna(current_classification):
            reason = "current classification is NaN"
            update_needed = True
        elif not isinstance(current_classification, str):
            reason = "current classification is not a string"
            update_needed = True
        elif current_classification.lower() not in allowed_classes:
            reason = "current classification is invalid"
            update_needed = True
        else:
            reason = "current classification is valid"
            update_needed = False

        print(f"Reason for update: {reason}")

        if update_needed:
            if new_classification in allowed_classes:
                df.loc[idx, 'classification'] = new_classification
            else:
                df.loc[idx, 'classification'] = 'NONE'
        else:
            error_indices.append(idx)
    
    # Fill any remaining NaNs in 'classification' column with 'NONE'
    df['classification'] = df['classification'].fillna('NONE')
    
    return df, error_indices



Detected CSV encoding: MacRoman
Detected log encoding: Windows-1252
Processing row 0: current classification = 'appropriate', new classification = 'appropriate'
Reason for update: current classification is valid
Processing row 1: current classification = 'inappropriate', new classification = 'appropriate'
Reason for update: current classification is valid
Processing row 2: current classification = 'appropriate', new classification = 'appropriate'
Reason for update: current classification is valid
Processing row 3: current classification = 'appropriate', new classification = 'appropriate'
Reason for update: current classification is valid
Processing row 4: current classification = 'nan', new classification = 'appropriate'
Reason for update: current classification is NaN
Processing row 5: current classification = 'inappropriate', new classification = 'inappropriate'
Reason for update: current classification is valid
Processing row 6: current classification = 'inappropriate', new classifi

In [ ]:
# Paths to the files
file_name = "EN_reply_comment_title_description_classify_replies_561_MIXTRAL_Hatebase_FINAL"
csv_directory = "C:/Users/aboeck/Documents/ICMT/MASTERARBEIT_DI/backup_03_07/EVALUATION/experiments_outputs/MIXTRAL/"
csv_file_path = os.path.join(csv_directory, f'{file_name}.csv')
log_file_unclean = f"C:/Users/aboeck/Documents/ICMT/MASTERARBEIT_DI/backup_03_07/EVALUATION/log_files/MIXTRAL/_{file_name}.log"
log_directory = "C:/Users/aboeck/Documents/ICMT/MASTERARBEIT_DI/backup_03_07/EVALUATION/log_files/MIXTRAL/cleaned_logs/"
log_file_name = f'_{file_name}_cleaned.log'
log_file_path = os.path.join(log_directory, log_file_name)

# Detect encodings
csv_encoding = detect_file_encoding(csv_file_path)
log_encoding = detect_file_encoding(log_file_path)

print(f"Detected CSV encoding: {csv_encoding}")
print(f"Detected log encoding: {log_encoding}")

# Read the CSV file
df = pd.read_csv(csv_file_path, sep=';', encoding=csv_encoding)

# Allowed classification values
allowed_classes = {"appropriate", "offensive", "violent", "inappropriate"}

# Extract log data
log_data = read_log_file(log_file_path, encoding=log_encoding)

# Update DataFrame with log data and capture error indices
updated_df, error_indices = update_dataframe_with_log_data(df, log_data, allowed_classes)

# Print error indices
if error_indices:
    print(f"NO update needed for indices: {error_indices}")

# Create the subfolder if it doesn't exist
subfolder = os.path.join(os.path.dirname(csv_file_path), 'cleaned')
os.makedirs(subfolder, exist_ok=True)

# Construct the output file path
output_csv_file_path = os.path.join(subfolder, os.path.basename(csv_file_path))

# Save the updated DataFrame to the new CSV file
updated_df.to_csv(output_csv_file_path, index=False, sep=';', encoding=csv_encoding)

print(f"Updated DataFrame saved to: {output_csv_file_path}")

# Set display options to show all rows and full text
pd.set_option('display.max_rows', None)  # Show all rows
